# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/IamCreateAI/Ruyi-Mini-7B',
 '/Datou1111/shou_xin',
 '/answerdotai/ModernBERT-base',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/tencent/HunyuanVideo',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/HuggingFaceH4/blogpost-scaling-test-time-compute',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/HuggingFaceTB/finemath',
 '/datasets/amphora/QwQ-LongCoT-130K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/do

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
IamCreateAI/Ruyi-Mini-7B
Updated
4 days ago
•
8.17k
•
352
Dat

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
print(get_brochure_user_prompt("HuggingFace", "https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}]}
You are looking at a company called: HuggingFace
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
IamCreateAI/Ruyi-Mini-7B
Updated
4 days ago
•
8.17k
•
352
Datou1111/shou_xin
Updated
12 days ago
•
28.3k
•
672
answerdotai/ModernBERT-base
Updated
1 day ago
•
6.24k
•
236
me

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.com/'}, {'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

**Hugging Face**  
*The AI community building the future.*

---

## About Us
Hugging Face is a pioneering platform where the machine learning community comes together to collaborate on models, datasets, and applications. With over 400,000 models and 100,000 datasets available, we empower users to create, discover, and innovate in the field of machine learning.

### Our Mission
To accelerate the development and deployment of machine learning applications, making cutting-edge technology accessible to everyone.

---

## Company Culture
At Hugging Face, we believe in the power of collaboration and open-source technology. We foster an inclusive environment where every team member's input is valued, allowing for diverse ideas and perspectives. Our culture emphasizes continuous learning, innovation, and a commitment to advancing AI for the greater good.

---

## Customers
Hugging Face serves more than 50,000 organizations, including industry leaders such as:

- **Amazon Web Services**
- **Meta**
- **Google**
- **Microsoft**
- **Intel**
  
These organizations utilize our platform for various machine learning tasks, enhancing their workflows and outputs.

---

## Careers at Hugging Face
We are always on the lookout for talented individuals who are passionate about AI and machine learning. Career opportunities at Hugging Face offer:

- A collaborative work environment
- Remote work flexibility
- Continuing education and mentorship
- Opportunities to work on impactful projects

**Join us and help shape the future of AI!**

---

## Our Offerings
### Models
- Access over 400,000 machine learning models, covering a variety of tasks and technologies.

### Datasets
- Discover and share 100,000+ datasets tailored for computer vision, audio, and NLP tasks.

### Spaces
- Utilize our application space to run various applications including real-time projects and demonstrations.

### Enterprise Solutions
- With dedicated support and industry-grade security, our Enterprise solutions are designed for organizations looking to implement AI at scale.

---

## Get Started Today!
**Sign up now** to become part of the Hugging Face community and access an array of tools to accelerate your machine learning journey.  
[Sign Up Here](#)

---

**Stay Connected**  
Follow us on our social media platforms:
- [GitHub](#)
- [Twitter](#)
- [LinkedIn](#)
- [Discord](#)

**Hugging Face – Building the Future of AI**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face

## The AI Community Building the Future

At Hugging Face, we bring together the machine learning community to collaborate on groundbreaking models, datasets, and applications. Our platform is a vibrant hub where innovation meets practicality, empowering developers and researchers to create state-of-the-art AI solutions.

---

### 🏆 What We Offer

- **Models**: Access and discover over **400k+ models** including the latest advancements in AI.
- **Datasets**: A rich collection of **100k+ datasets** tailored for various machine learning tasks.
- **Spaces**: Collaborate on applications and projects seamlessly within our community’s creative workspace.

---

### 🌏 Our Customers

Join the ranks of **50,000+ organizations** leveraging Hugging Face’s offerings, including industry giants like:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Grammarly**

These companies trust us to accelerate their machine learning initiatives and foster innovation.

---

### 🌱 Company Culture

At Hugging Face, we embrace an open-source ethos, encouraging collaboration and contribution from the community. Our culture is centered around creativity, innovation, and inclusivity. We believe in empowering individuals and teams by providing the right tools and support to shape the future of AI.

---

### 🚀 Careers at Hugging Face

We are on the lookout for passionate individuals to join our team! If you share our vision of an accessible AI landscape, explore the career opportunities we offer. We provide an environment that supports academic growth, teamwork, and professional development while making a meaningful impact in the machine learning realm.

#### Current Openings Include:
- Machine Learning Engineers
- Data Scientists
- Software Developers
- Community Managers

---

### 💡 Join Us

Are you ready to be part of a revolution in AI? **[Sign Up](#)** today to explore the possibilities with Hugging Face or **[Log In](#)** if you’re already part of our community.

Let’s build the future of AI together!

---

*For inquiries about our enterprise solutions, pricing, or community involvement, feel free to reach out through our website.* 

**Connect with us:**  
[Twitter](#) | [LinkedIn](#) | [GitHub](#) | [Forum](#)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face: The AI Community Building the Future

Welcome to Hugging Face, the leading collaborative platform for the machine learning community. With a robust environment designed for creating, discovering, and deploying machine learning models, datasets, and applications, Hugging Face is at the frontier of artificial intelligence innovation. 

---

## About Us
At Hugging Face, we believe in the power of collaboration. Our platform enables users to work together on projects that range from machine-learning models to expansive datasets. With over 400,000 models and 100,000 datasets available, we provide the tools necessary to help researchers, developers, and organizations accelerate their machine learning projects.

- **Trending Models This Week:**
  - **IamCreateAI/Ruyi-Mini-7B** | 8.17k | Updated 4 days ago
  - **Datou1111/shou_xin** | 28.3k | Updated 12 days ago
  - **meta-llama/Llama-3.3-70B-Instruct** | 236k | Updated 11 days ago

Explore our community-driven approach that integrates state-of-the-art tools like Transformers, DiffUsers, and PEFT (Parameter Efficient Finetuning).

---

## Company Culture
Hugging Face fosters a vibrant and inclusive company culture, aiming to empower individuals and teams through transparent practices and open-source methodologies. We believe in “AI for everyone,” promoting accessibility and co-creation within the AI community. 

### Why Work With Us?
- **Collaborative Environment**: Join a diverse team of experts and enthusiasts dedicated to pushing the boundaries of AI and machine learning.
- **Open Source Commitment**: Contribute to freely accessible tools that serve the global community.
- **Flexible Work**: We support remote work and provide a range of job opportunities tailored to different areas of expertise.

---

## Customers & Organizations
Over 50,000 organizations utilize Hugging Face in various industries, including notable names such as:
- **Meta AI**
- **Amazon Web Services**
- **Google**
- **Microsoft**

Our enterprise solutions offer seamless integration with advanced security features, making us a trusted partner for both startups and established corporations.

---

## Careers at Hugging Face
We are always on the lookout for passionate individuals to join our team. Explore our open positions in areas such as software development, research, marketing, and customer support.

- **Open Positions**: 
  - Machine Learning Engineer
  - Data Scientist
  - Community Manager

Join us in shaping the future of AI. 

**[Explore Careers](#)**

---

## Join the Hugging Face Community
Whether you're looking to develop cutting-edge AI models, contribute to open-source projects, or advance your career in this dynamic field, Hugging Face is your gateway to innovation.

**[Learn More](#)** | **[Sign Up Today](#)**

Together, let's build the future of AI!



## **Multi-lingual with Desire Format**


In [24]:
def multi_lingual_stream_brochure(company_name, url, language, tone):

    system_prompt = f"""
You are an assistant that analyzes the contents of several relevant pages from a company website and creates a visually appealing and professional short brochure for prospective customers, investors, and recruits. 
The brochure should be written in {language} and use a {tone.lower()} tone throughout.

The brochure should follow this structure (in {language}):

1. **Front Cover**:
   - Prominently display the company name as Title.
   - Include a compelling headline or tagline.
   - Add something engaging relevant to the company’s mission.

2. **About Us**:
   - Provide a brief introduction to the company.
   - State the company’s core mission and vision.
   - Mention the founding story or key milestones.

3. **What We Offer**:
   - Summarize the company's products, services, or solutions.
   - Highlight benefits or unique selling points.
   - Include testimonials or case studies if available.

4. **Our Culture**:
   - Outline the company’s key values or guiding principles.
   - Describe the workplace environment (e.g., innovation-driven, inclusive, collaborative).
   - Highlight community engagement or CSR initiatives.

5. **Who We Serve**:
   - Describe the target customers or industries served.
   - Mention notable clients or partners.
   - Include testimonials or endorsements from customers.

6. **Join Us**:
   - Detail career or internship opportunities.
   - Highlight benefits, career growth, or training opportunities.
   - Provide direct links or steps to apply.

7. **Contact Us**:
   - Provide the company’s address, phone number, and email.
   - Include links to social media platforms.
   - Add a link to the company’s website.

8. **Closing Note**:
   - End with a thank-you message or an inspirational note for the reader.
   - Add a call-to-action (e.g., “Get in touch today!” or “Explore more on our website”).

Ensure the content is concise, engaging, visually clear, and tailored to the target audience. Use headings and subheadings to make the brochure easy to navigate. Include links and contact information wherever applicable.
"""


    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:

multi_lingual_stream_brochure("OpenAI", "https://openai.com/", "Urdu", "humorous, entertaining, jokey")

Found links: {'links': [{'type': 'about page', 'url': 'https://openai.com/about'}, {'type': 'careers page', 'url': 'https://openai.com/careers'}]}


It seems that the landing and related pages for OpenAI did not yield any specific content. However, I can create a creative and engaging brochure based on general knowledge about OpenAI. Here's a humorous and entertaining brochure written in Urdu:


# 🎉 اوپن اے آئی: ہوشیار robots کا دوست! 🎉

---

## About Us - ہمارے بارے میں:

ہماری کمپنی اوپن اے آئی، 2015 میں بنی۔ ہم نے سوچا کہ "کیوں نہ ایک ایسا انٹیلیجنٹ سسٹم بنائیں جو انسانوں کی مدد کرے؟" تو ہم نے کام شروع کیا اور دیکھیں! ہم نے ایک نئی دنیا کی بنیاد رکھی۔ ہماری مشن ہے "تمام لوگوں کے لئے AI کی طاقت کو قابل رسائی بنانا"۔ آفاقی طاقت کو ڈھونڈتے ہیں، جیسے آپ کے فرج میں چھپے ہوئے برگر!

---

## What We Offer - ہم کیا پیش کرتے ہیں:

ہم AI کے شوقین ہیں! 🤖 ہم مختلف پروڈکٹس اور سروسز پیش کرتے ہیں، جیسے کہ:

- **GPT-3**: آپ کے سوالات کے جواب دینے کے لئے تیار!
- **تخلیقی تحریر**: جنریٹنگ آئیڈیاز جب آپ کی تخلیقیت بریک ہو جائے!
- **AI ٹولز**: آپ کی زندگی کو مزید آسان بنانے کے لئے!

ہمارے صارفین کہتے ہیں، "اپنی زندگی میں اوپن اے آئی کی ضرورت ہے، جیسے موٹیویشن کی ضرورت ہوتی ہے!"

---

## Our Culture - ہماری ثقافت:

ہماری کمپنی میں، ہمارا بنیادی اصول ہے: "پیار اور انوکھا خیالات!" 🤗 ہم نے انوکھے، تعاون پر مبنی ماحول کی بنیاد رکھی، جہاں ہر کوئی اپنی بات کہہ سکتا ہے، یہاں تک کہ ونڈو کے باہر کھڑا درخت بھی! ہم کمیونٹی کی خدمت کیلئے ہمیشہ تیار رہتے ہیں، وہ بھی سوشل میڈٰیا پر۔

---

## Who We Serve - ہم کس کی خدمت کرتے ہیں:

ہم ہر اُس شخص کی خدمت کرتے ہیں جو سوپر ہیرومنٹ کی تلاش میں ہے۔ ہمارے وزیٹر، محققین، اور ٹیکنالوجی کے شوقین ہیں، اور ہمارے بہترین کلائنٹس include شامل ہیں "بڑا دماغی جیسا سوچنے والے!" 💡

---

## Join Us - ہمارے ساتھ شامل ہوں:

آپ کو ترقی کی تلاش ہے؟ تو ہماری ٹیم کا حصہ بنیں! 🚀 ہم ہمیشہ نئے امریکی جاموں کی تلاش میں ہیں۔ آپ کو ٹریننگ، ترقی کے مواقع، اور سہولیات فراہم کریں گے۔

📩 **درخواست دینے کے مرحلے:** ہماری ویب سائٹ پر جائیں، کیونکہ ہم جانتے ہیں کہ آپ کا خواب آپ کے قریب ہے!

---

## Contact Us - ہم سے رابطہ کریں:

**پتہ:** نیٹ ورک کی دنیا  
**فون:** 123-456-789  
**ایمیل:** info@openai.com  
**سوشل میڈیا:** [فیس بک](#) | [ٹویٹر](#) | [لنکڈ ان](#)  
**ویب سائٹ:** [openai.com](#)

---

## Closing Note - اختتامی نوٹ:

ہماری کمپنی اوپن اے آئی کی طرف سے ایک شکریہ! اے آئی کی دنیا میں قدم رکھنے کا وقت آ گیا ہے! 

🖱️ **آج ہی رابطہ کریں یا ہماری ویب سائٹ کا دورہ کریں!**


**نوٹ:** واقعی ویب سائٹ کے مخصوص روابط، ای میل اور نمبر تخلیقی مقصد کے لئے ہیں۔ اس کو حقیقی معلومات کے ساتھ تبدیل کیا جا سکتا ہے۔